In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pandas as pd
import math

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import xgboost

from sklearn.model_selection import train_test_split


In [4]:
df= pd.read_csv('data/train.csv', low_memory=False)

In [5]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [6]:
#For Product category create 20 new features.

In [67]:
df.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [8]:
df.Product_Category_2.unique()

array([nan,  6., 14.,  2.,  8., 15., 16., 11.,  5.,  3.,  4., 12.,  9.,
       10., 17., 13.,  7., 18.])

In [9]:
df.Product_Category_3.unique()

array([nan, 14., 17.,  5.,  4., 16., 15.,  8.,  9., 13.,  6., 12.,  3.,
       18., 11., 10.])

In [10]:
#convert each factor into integer and '4+'=4

In [11]:
df.Stay_In_Current_City_Years.unique()

array(['2', '4+', '3', '1', '0'], dtype=object)

In [12]:
#convert three features for City_Category (A,B,C)

In [13]:
df.City_Category.unique()

array(['A', 'C', 'B'], dtype=object)

In [14]:
df.Occupation.unique()     #Not yet decided. Let it be as it is.

array([10, 16, 15,  7, 20,  9,  1, 12, 17,  0,  3,  4, 11,  8, 19,  2, 18,
        5, 14, 13,  6])

In [15]:
df.Age.unique()    #group them into ordinal catogary

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [16]:
df.Gender.unique() #convert into 0 or 1

array(['F', 'M'], dtype=object)

In [17]:
df.head().T

,0,1,2,3,4
User_ID,1000001,1000001,1000001,1000001,1000002
Product_ID,P00069042,P00248942,P00087842,P00085442,P00285442
Gender,F,F,F,F,M
Age,0-17,0-17,0-17,0-17,55+
Occupation,10,10,10,10,16
City_Category,A,A,A,A,C
Stay_In_Current_City_Years,2,2,2,2,4+
Marital_Status,0,0,0,0,0
Product_Category_1,3,1,12,12,8
Product_Category_2,NaN,6,NaN,14,NaN


In [18]:
df.Product_ID.unique()

array(['P00069042', 'P00248942', 'P00087842', ..., 'P00370293',
       'P00371644', 'P00370853'], dtype=object)

In [19]:
df.User_ID.unique()

array([1000001, 1000002, 1000003, ..., 1004113, 1005391, 1001529])

In [20]:
len(set(df.User_ID.unique()))

5891

In [21]:
len(set(df.Product_ID.unique()))

3631

In [22]:
df.shape

(550068, 12)

In [23]:
dt = pd.read_csv("data/test.csv", low_memory=False)

In [24]:
dt.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3'],
      dtype='object')

In [25]:
dp = dt[["User_ID","Product_ID"]]

In [26]:
len(set(dt.User_ID.unique()))

5891

In [27]:
dt.shape

(233599, 11)

In [28]:
len(set(dt.Product_ID.unique()))

3491

In [29]:
#As we have Same number of User_ID So we will use the factor of it.
#But as have product_catogary and also less number of unique product id in test data so we remove this feature

In [30]:
df.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [31]:
#Drop Purchase value 
purchase = df["Purchase"]
df_train =df.drop("Purchase", axis=1)

In [32]:
df_train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN


In [33]:
df_train =df_train.drop("Product_ID", axis=1)  #drop the Product ID
df_test =dt.drop("Product_ID", axis=1)  #drop the Product ID

In [34]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_train["Gender"] = labelencoder.fit_transform(df_train["Gender"])
df_test["Gender"] = labelencoder.fit_transform(df_test["Gender"])

In [35]:
#try with age group encoding in ordinal fassion
df_train["Age"] = labelencoder.fit_transform(df_train["Age"])
df_test["Age"] = labelencoder.fit_transform(df_test["Age"])

In [36]:
#try with Stay_In_current_city group encoding in ordinal fassion
df_train["Stay_In_Current_City_Years"] = labelencoder.fit_transform(df_train["Stay_In_Current_City_Years"])
df_test["Stay_In_Current_City_Years"] = labelencoder.fit_transform(df_test["Stay_In_Current_City_Years"])

In [37]:
df.City_Category.unique()

array(['A', 'C', 'B'], dtype=object)

In [38]:
df_test.head()

,User_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,1,4,7,B,2,1,1,11.0,NaN
1,1000009,1,2,17,C,0,0,3,5.0,NaN
2,1000010,0,3,1,B,4,1,5,14.0,NaN
3,1000010,0,3,1,B,4,1,4,9.0,NaN
4,1000011,0,2,1,C,1,0,4,5.0,12.0


In [39]:
dff=pd.get_dummies(df_train["City_Category"])
dtt=pd.get_dummies(df_test["City_Category"])

In [40]:
df_c = pd.concat([dff.reset_index(drop=True), df_train], axis=1)
df_t = pd.concat([dtt.reset_index(drop=True), df_test], axis=1)

In [41]:
df_t.head()

,A,B,C,User_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,0,1,0,1000004,1,4,7,B,2,1,1,11.0,NaN
1,0,0,1,1000009,1,2,17,C,0,0,3,5.0,NaN
2,0,1,0,1000010,0,3,1,B,4,1,5,14.0,NaN
3,0,1,0,1000010,0,3,1,B,4,1,4,9.0,NaN
4,0,0,1,1000011,0,2,1,C,1,0,4,5.0,12.0


In [42]:
df_c = df_c.drop("City_Category",axis=1)
df_t = df_t.drop("City_Category",axis=1)

In [43]:
df_c.Product_Category_1.unique()

array([ 3,  1, 12,  8,  5,  4,  2,  6, 14, 11, 13, 15,  7, 16, 18, 10, 17,
        9, 20, 19])

In [44]:
val = "Product_Category_"
tt = pd.DataFrame()
ll= [0]*len(df_t)
for i in range(1,21):
    tt[val+str(i)]=ll

In [45]:
val = "Product_Category_"
dd = pd.DataFrame()
list= [0]*len(df_c)
for i in range(1,21):
    dd[val+str(i)]=list
    

In [46]:
#train data
for i in range(len(df_c["Product_Category_1"])):
    cat = df_c["Product_Category_1"][i]
    dd[val+str(cat)][i]=1


In [47]:
#test data
for i in range(len(df_t["Product_Category_1"])):
    cat = df_t["Product_Category_1"][i]
    tt[val+str(cat)][i]=1 

In [48]:
#train data
for i in range(len(df_c["Product_Category_2"])):
    cat = df_c["Product_Category_2"][i]
    if math.isnan(cat) == False:
        dd[val+str(int(cat))][i]=1

In [49]:
#test data
for i in range(len(df_t["Product_Category_2"])):
    cat = df_t["Product_Category_2"][i]
    if math.isnan(cat) == False:
        tt[val+str(int(cat))][i]=1

In [50]:
#train data
for i in range(len(df_c["Product_Category_3"])):
    cat = df_c["Product_Category_3"][i]
    if math.isnan(cat) == False:
        dd[val+str(int(cat))][i]=1

In [51]:
#test data
for i in range(len(df_t["Product_Category_3"])):
    cat = df_t["Product_Category_3"][i]
    if math.isnan(cat) == False:
        tt[val+str(int(cat))][i]=1

In [52]:
#drop some of the columns
df_c = df_c.drop(["Product_Category_1","Product_Category_2","Product_Category_3"],axis=1)
df_t = df_t.drop(["Product_Category_1","Product_Category_2","Product_Category_3"],axis=1)

In [53]:
#combine both the dataframe
df_pro = pd.concat([dd.reset_index(drop=True), df_c], axis=1)
df_tet = pd.concat([tt.reset_index(drop=True), df_t], axis=1)

In [54]:
df_pro.head()

,Product_Category_1,Product_Category_2,Product_Category_3,Product_Category_4,Product_Category_5,Product_Category_6,Product_Category_7,Product_Category_8,Product_Category_9,Product_Category_10,...,Product_Category_20,A,B,C,User_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status
0,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1000001,0,0,10,2,0
1,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1000001,0,0,10,2,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1000001,0,0,10,2,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1000001,0,0,10,2,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1000002,1,6,16,4,0


In [55]:
df_tet.columns

Index(['Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Product_Category_4', 'Product_Category_5', 'Product_Category_6',
       'Product_Category_7', 'Product_Category_8', 'Product_Category_9',
       'Product_Category_10', 'Product_Category_11', 'Product_Category_12',
       'Product_Category_13', 'Product_Category_14', 'Product_Category_15',
       'Product_Category_16', 'Product_Category_17', 'Product_Category_18',
       'Product_Category_19', 'Product_Category_20', 'A', 'B', 'C', 'User_ID',
       'Gender', 'Age', 'Occupation', 'Stay_In_Current_City_Years',
       'Marital_Status'],
      dtype='object')

In [56]:
df_tet.head().T

,0,1,2,3,4
Product_Category_1,1,0,0,0,0
Product_Category_2,0,0,0,0,0
Product_Category_3,0,1,0,0,0
Product_Category_4,0,0,0,1,1
Product_Category_5,0,1,1,0,1
Product_Category_6,0,0,0,0,0
Product_Category_7,0,0,0,0,0
Product_Category_8,0,0,0,0,0
Product_Category_9,0,0,0,1,0
Product_Category_10,0,0,0,0,0


In [57]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df_pro,purchase, test_size=0.01)

In [58]:
#BASE MODEL

def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), Y_train), rmse(m.predict(X_valid), Y_valid), m.score(X_train,Y_train), m.score(X_valid,Y_valid)]
    
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [59]:
X_train.shape, Y_train.shape,X_valid.shape

((544567, 29), (544567,), (5501, 29))

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, Y_train)
print_score(m)

In [ ]:
#Building a single tree
m = RandomForestRegressor(n_estimators = 1, max_depth=3, bootstrap=False,n_jobs=-1)

m.fit(X_train, Y_train)
print_score(m)

In [ ]:
#Building a single tree
m = RandomForestRegressor(n_estimators = 40, max_depth=20,n_jobs=-1)

m.fit(X_train, Y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train,Y_train)
print_score(m)

In [ ]:
import numpy as np
preds = np.stack([t.predict(X_valid) for t in m.estimators_]) 
preds[:,0]

In [ ]:
preds.shape

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
plt.plot([metrics.r2_score(Y_valid, np.mean(preds[:i+1], axis=0)) for i in range(25)]);

In [ ]:
#Max_feature and min_feature

m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)

%time m.fit(X_train,Y_train)

print_score(m)

#Regression algorithm
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
m = reg.fit(X_train, Y_train)

In [60]:
#Try XGBOOST algorithm
# Let's try XGboost algorithm to see if we can get better results
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [61]:
m = xgb.fit(X_train, Y_train)

In [66]:
print_score(m)

[2930.0741736691207, 2954.500050279812, 0.6596751307605017, 0.6597213323140967]


In [62]:
predt = m.predict(df_tet)

In [63]:
predt = predt.astype(int)

In [64]:
dp["Purchase"]= predt

/home/geekkishor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
#convert dataframe into csv file
dp.to_dense().to_csv("submission.csv", index = False, sep=',', encoding='utf-8')